In [1]:
# Run if using tensorflow2.0+
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split

In [2]:
df_train = pd.read_csv('../Data/train.csv')
df_train['binary_open_channels'] = [int(x != 0) for x in df_train.open_channels]
df_train

,time,signal,open_channels,binary_open_channels
0,0.0001,-2.7600,0,0
1,0.0002,-2.8557,0,0
2,0.0003,-2.4074,0,0
3,0.0004,-3.1404,0,0
4,0.0005,-3.1525,0,0
...,...,...,...,...
4999995,499.9996,2.9194,7,1
4999996,499.9997,2.6980,7,1
4999997,499.9998,4.5164,8,1
4999998,499.9999,5.6397,9,1


In [3]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train.binary_open_channels)

data = np.zeros((len(signal), 2))
data[:, 0] = signal
data[:, 1] = open_channels
data = data.reshape((1000, 5000, 2))
np.random.shuffle(data)

x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1].reshape((1000,5000,1))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = .3)

In [17]:
model = Sequential()
model.add(SimpleRNN(units=20, return_sequences=True, input_shape=(5000,1,)))
model.add(Dense(20,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [18]:
model.fit(x_train, y_train, epochs=20, validation_data=(x_test,y_test),verbose=1)

Train on 700 samples, validate on 300 samples
Epoch 1/20
700/700 [==============================] - 55s 79ms/sample - loss: 0.4975 - accuracy: 0.8286 - val_loss: 0.4431 - val_accuracy: 0.8221
Epoch 2/20
700/700 [==============================] - 55s 78ms/sample - loss: 0.4058 - accuracy: 0.8405 - val_loss: 0.3889 - val_accuracy: 0.8287
Epoch 3/20
700/700 [==============================] - 55s 78ms/sample - loss: 0.3662 - accuracy: 0.8442 - val_loss: 0.3616 - val_accuracy: 0.8342
Epoch 4/20
700/700 [==============================] - 74s 106ms/sample - loss: 0.3439 - accuracy: 0.8486 - val_loss: 0.3434 - val_accuracy: 0.8391
Epoch 5/20
700/700 [==============================] - 58s 83ms/sample - loss: 0.3285 - accuracy: 0.8519 - val_loss: 0.3301 - val_accuracy: 0.8447
Epoch 6/20
700/700 [==============================] - 55s 78ms/sample - loss: 0.3172 - accuracy: 0.8561 - val_loss: 0.3192 - val_accuracy: 0.8492
Epoch 7/20
700/700 [==============================] - 55s 78ms/sample - loss:

In [19]:
predictions = model.predict(x)

In [20]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = y[i][j][0]
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
        elif actual == 0 and pred == 1:
            fp += 1
        elif actual == 1 and pred == 0:
            fn += 1
        else:
            tp += 1
total = tp+tn+fp+fn

In [21]:
print("Accuracy:",(tp+tn)*100/total)
print("True Positive Rate:",tp*100/total)
print("True Negative Rate:",tn*100/total)
print("False Postiive Rate:",fp*100/total)
print("False Negative Rate:",fn*100/total)

Accuracy: 94.41892
True Positive Rate: 74.20524
True Negative Rate: 20.21368
False Postiive Rate: 4.58936
False Negative Rate: 0.99172


In [22]:
sequence_acc = []
for i in range(len(y)):
    correct = 0
    incorrect = 0
    for j in range(len(y[i])):
        actual = y[i][j][0]
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
            correct += 1
        elif actual == 0 and pred == 1:
            fp += 1
            incorrect += 1
        elif actual == 1 and pred == 0:
            fn += 1
            incorrect += 1
        else:
            tp += 1
            correct += 1
    sequence_acc.append(correct/(correct+incorrect))

In [23]:
print("Minimum acc in chunk:", min(sequence_acc))
count = 0
for val in sequence_acc:
    if val == 1.0:
        count += 1
print("Number of perfect chunks:", (count))

Minimum acc in chunk: 0.0
Number of perfect chunks: 146


In [24]:
for i in range(len(sequence_acc)):
    val = sequence_acc[i]
    if val == 0.0:
        print(i)

357


In [16]:
x_357 = []
y_357 = []
pred_357 = []
for i in range(1000):
    x_357.append(x[357][i][0])
    y_357.append(y[357][i][0])
    pred_357.append(predictions[357][i][0])
df_analysis = pd.DataFrame()
df_analysis['Signal'] = x_357
df_analysis['Actual'] = y_357
df_analysis['Predicted'] = pred_357
df_analysis

,Signal,Actual,Predicted
0,0.2733,0.0,0.689898
1,0.3884,0.0,0.810469
2,-0.1229,0.0,0.784324
3,-0.1870,0.0,0.764345
4,-0.1912,0.0,0.821175
...,...,...,...
995,-0.2777,0.0,0.719673
996,-0.2788,0.0,0.682233
997,-0.1582,0.0,0.723067
998,0.0817,0.0,0.766391
